<a href="https://colab.research.google.com/github/Alyona-Stankova/Method_micromobility/blob/main/Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Подготовка к использованию метода
Метод позволяет работать как с собственным файлом, так и с данными из OpenStreetMap.

Выберете **внутри разделов** (сервисы по площади и сервисы по вместимости) конкртные серисы:

1) для **сервисов по площади** доступны: бизнес-центры, торговые центры;

2) для **сервисов по вместимости** доступны: школы, медецинские учреждения, культурно-досуговые центры.
По умолчанию в коде прописаны бизнес-центры и школы соответсвенно.

Внутри кода имеются текстовые сноски с написанной частью кода, на которую небходимо заменить для работы с другими сервисами из списка. Кроме того, в коде имеются сноски и рекомендации по замене наименований переменных для корректной работы кода.

**Важно!**

Работа с **собственным файлом**:

При работе с собственным файлом (в формате geojson) важно наличие следующих столбцов в атрибутивной таблице геослоя:

- для **жилых зданий**: парковочные_места_СИМ, зарядные_места_СИМ, количество_квартир, addr:housenumber, addr:street;
- для **сервисов по площади**: парковочные_места_СИМ, зарядные_места_СИМ, Площадь, addr:housenumber, addr:street;
- для **сервисов по вместимости**: парковочные_места_СИМ, зарядные_места_СИМ, Вместимость, addr:housenumber, addr:street, name;
- для **остановок наземного транспорта**: парковочные_места_СИМ, зарядные_места_СИМ;
- для **станций метро**: парковочные_места_СИМ, зарядные_места_СИМ.

Для корректной работы метода и увеличения точноти расчётов рекомендуется брать данные о количестве квартир, площади и вместимости из надежных и открытых источников.

Все столбцы с атрибутами объектов должны быть заполнены, пропуски или нулевые значения допускаются только в столбцах: парковочные_места_СИМ, зарядные_места_СИМ.

Информация об атрибутах: addr:housenumber, addr:street, name есть в OpenStreetMap.

Работа с **данными из OpenStreetMap**:

При выгрузке данных из OpenStreetMap важно корректно указать наименование территории и тэги.

Дополнительная работа с данными из OSM не требуется, неободимые преобразования данных предусмотрены в коде. Важно отметить, что данные из OSM не обладают высокой точностью, в связи с чем в полученных данным могут быть неточности.

**Внимательно** читайте файл Readme и иснструкции в внутри разделов кода!

In [430]:
 # перед началом использования кода:
result_service=0
count_service=0

# Загрузка модулей

Загрузите все библиотеки и модули, представленные в этом разделе. Это необходимо для корректной работы кода в дальнейшем

In [181]:
!pip install blocksnet ipykernel -q

In [182]:
from blocksnet import BlocksGenerator
from blocksnet import City

In [183]:
!pip install mapclassify -q

In [184]:
import osmnx as ox
import geopandas as gpd
import pandas as pd

import os
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from shapely.geometry import Point
import folium
from shapely.geometry import mapping


# Жилые здания

**Загрузка данных**

Выберите **один** из способов загрузки данных:
1. Данные из OpenStreetMap (OSM) - **вставьте название** своей территории в переменную "area";
2. Собственный файл - подгрузите Ваш файл в Jupyter Notebook, проверьте наличие **всех необходимых столбцов** в Вашем файле и обратите внимание на **название файла**, указанного в коде (либо назавите свой файл идентично, либо поменяйте название внутри кода)

In [431]:
 # укажите свою территорию в переменной area
area = "Московский район, Санкт-Петербург, Россия"
buildings = ox.geometries_from_place(area, tags={'building': True})

In [391]:
  # подгрузить файл с подготовленный жилыми зданиями с полями "парковочные_места_СИМ", "зарядные_места_СИМ", "количество_квартир", "addr:housenumber", "addr:street"
  # название Вашего файла и название файла в коде должны совпадать
buildings = gpd.read_file('/content/buildings_over.geojson')

In [ ]:
buildings # проверка данных

In [ ]:
 # картографическое представление данных
buildings.explore()

**Метод**

Запустите последовательно все части кода внутри подраздела "Метод".
Убедитесь, что таблица атрибутов измениилась (или появилась информация о достаточности инфраструктуры для СИМ)

In [432]:
def calculate_lots(geos_layer, result, count):
    # проверяем наличие необходимых атрибутов
    if all(attr in geos_layer.columns for attr in ['парковочные_места_СИМ', 'зарядные_места_СИМ', 'количество квартир']):

              # проверяем минимальное необходимое количество мест
            required_parking = round(geos_layer['количество квартир'] * 0.8)
            required_charging = round(required_parking * 0.1)
              # проверяем, более 50% объектов удовлетворяют условиям оценки
            satisfied_count_parking = (geos_layer['парковочные_места_СИМ'] >= required_parking).sum()
            satisfied_count_charging = (geos_layer['зарядные_места_СИМ'] >= required_charging).sum()
            total_count = len(geos_layer)

            parking_ratio = satisfied_count_parking / total_count
            charging_ratio = satisfied_count_charging / total_count

            if parking_ratio >= 0.5 and charging_ratio >= 0.5:
               result += 1
               count += 1
            elif parking_ratio <= 0.5 and charging_ratio >= 0.5:
               result += 0.5
               count += 1
            elif parking_ratio >= 0.5 and charging_ratio <= 0.5:
               result += 0.5
               count += 1
            else:
               count += 1

            print(f"Результат сервиса: {result}")
            print(f"Количество пройденных сервисов на территории: {count}")

              # обновляем парковочные места только если текущее значение меньше требуемого
            if (geos_layer['парковочные_места_СИМ'] < required_parking).any():
              geos_layer.loc[geos_layer['парковочные_места_СИМ'] < required_parking, 'парковочные_места_СИМ'] = required_parking
            else:
               print("Достаточное количество парковочных мест")

             # добавляем новое условие для проверки избыточности парковочных мест
            if ((geos_layer['парковочные_места_СИМ']) > required_parking + 20).any():
                print("Предупреждение: возможно избыточное количество парковочных мест")

             # обновляем зарядные места только если текущее значение меньше требуемого
            if (geos_layer['зарядные_места_СИМ'] < required_charging).any():
                geos_layer.loc[geos_layer['зарядные_места_СИМ'] < required_charging, 'зарядные_места_СИМ'] = required_charging
            else:
                print("Достаточное количество зарядных мест")

             # добавляем новое условие для проверки избыточности зарядных мест
            if ((geos_layer['зарядные_места_СИМ']) > required_charging + 15).any():
                  print("Предупреждение: возможно избыточное количество зарядных мест")

    else:
        # обработка геослоя
        geos_layer = geos_layer[geos_layer.geometry.type.isin(['Polygon', 'MultiPolygon'])]
        geos_layer = geos_layer.reset_index(drop=True)
        original_crs = geos_layer.crs
        local_crs = geos_layer.estimate_utm_crs()  # определяем локальную систему координат
        geos_layer = geos_layer.to_crs(local_crs)  # переводим здания в локальную систему координат

        # преобразуем столбец 'building:levels' в числовой тип, ошибки будут заменены на NaN
        geos_layer['building:levels'] = pd.to_numeric(geos_layer['building:levels'], errors='coerce')
        # заполним пропущенные значения (NaN) нулями
        geos_layer = geos_layer.fillna(0)

        # добавляем или преобразуем необходимые атрибуты
        # 1. количество этажей (number_of_floors)
        geos_layer['number_of_floors'] = geos_layer.apply(
            lambda x: max(1, x['building:levels']), axis=1
        )

        # 2. площадь застройки (footprint_area) - как площадь геометрии (основание здания)
        geos_layer['footprint_area'] = geos_layer.geometry.area

        # 3. общая площадь всех этажей (build_floor_area) - footprint_area * number_of_floors
        geos_layer['build_floor_area'] = geos_layer['footprint_area'] * geos_layer['number_of_floors']

        # 4. жилая площадь (living_area)
        residential_tags = ['residential', 'house', 'apartments', 'detached', 'terrace', 'dormitory']
        geos_layer['living_area'] = geos_layer.apply(
            lambda x: 0.8 * x['build_floor_area'] if x.get('building') in residential_tags else 0,
            axis=1
        )

        # нежилая площадь будет 20% от общей площади этажей
        geos_layer['non_living_area'] = geos_layer['build_floor_area'] - geos_layer['living_area']

        # 5. население (population) жилых зданий
        geos_layer['population'] = geos_layer.apply(
            lambda x: 48 * x['number_of_floors'] if x.get('building') in residential_tags else 0,
            axis=1
        )

        # удаляем здания без жилой площади
        geos_layer['living_area'] = pd.to_numeric(geos_layer['living_area'], errors='coerce')
        geos_layer = geos_layer.loc[geos_layer['living_area'] != 0]

        # расчёт количества квартир
        geos_layer['количество квартир'] = geos_layer['population'] / 4

        # сохраняем только необходимые столбцы
        geos_layer = geos_layer[['addr:street', 'addr:housenumber', 'geometry', 'build_floor_area',
                                  'living_area', 'footprint_area', 'number_of_floors',
                                  'population', 'количество квартир']]

        # расчёт парковочных и зарядных мест
        geos_layer['парковочные_места_СИМ'] = round(geos_layer['количество квартир'] * 0.8)
        geos_layer['зарядные_места_СИМ'] = round(geos_layer['парковочные_места_СИМ'] * 0.1)

        # возвращаем к оригинальной системе координат
        geos_layer = geos_layer.to_crs(original_crs)
    return geos_layer, result, count

In [ ]:
buildings, result_service, count_service = calculate_lots(buildings, result_service, count_service)
buildings

**Сохранение файла**

In [ ]:
buildings.to_file('buildings_osm_result.geojson')

**Визуализация**

Запустите последовательно код в подразделе "Визуализация".

Дизайн карты (подложка, размер круговых маркеров, цвета объектов и т.д.) можно изменить по желанию.

In [ ]:
buildings['centroid'] = buildings.geometry.centroid

# добавление столбцов для широты и долготы центроидов
buildings['latitude'] = buildings['centroid'].y
buildings['longitude'] = buildings['centroid'].x

# удаление временного столбца 'centroid', если он не нужен
buildings = buildings.drop(columns='centroid')

In [ ]:
# создание DataFrame
df = pd.DataFrame(buildings)

# используем средние значения для центра карты
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=12)

# создаем группы для парковочных и зарядных мест
parking_group = folium.FeatureGroup(name='Парковочные места')
charging_group = folium.FeatureGroup(name='Зарядные места')
buildings_group = folium.FeatureGroup(name='Жилые здания')

folium.TileLayer('CartoDB positron').add_to(m)

# итерация по строкам DataFrame
for idx, row in df.iterrows():
    # проверяем наличие необходимых колонок
    if 'парковочные_места_СИМ' in row and 'зарядные_места_СИМ' in row:
        # размер точки на основе количества парковочных мест
        parking_size = row['парковочные_места_СИМ'] / 9  # измените делитель по необходимости
        charging_size = row['зарядные_места_СИМ'] / 7  # измените делитель по необходимости

        # добавляем круговые маркеры для парковочных мест
        parking_group.add_child(
            folium.CircleMarker(
                location=(row['latitude'], row['longitude']),  # используем координаты текущей строки
                radius=parking_size,
                color=None,
                fill=True,  # можно изменить дизайн
                fill_color='#BFA181',  # можно изменить дизайн
                fill_opacity=0.6,  # можно изменить дизайн
                tooltip=f"Парковочные места: {row['парковочные_места_СИМ']}"
            )
        )

        # добавляем круговые маркеры для зарядных мест
        charging_group.add_child(
            folium.CircleMarker(
                location=(row['latitude'], row['longitude']),  # используем координаты текущей строки
                radius=charging_size,
                color=None,
                fill=True,  # можно изменить дизайн
                fill_color='#0A1828',  # можно изменить дизайн
                fill_opacity=0.9,  # можно изменить дизайн
                tooltip=f"Зарядные места: {row['зарядные_места_СИМ']}"  # исправлено на нужное поле
            )
        )

# итерация для добавления зданий на карту
for index, row in df.iterrows():
    # п наличие геометрии
    if 'geometry' in row and row['geometry'] is not None:
        geometry = row['geometry']

        # одбавляем полигональный слой зданий на карту
        buildings_group.add_child(
            folium.GeoJson(
                geometry,  # используем всю геометрию
                color=None,
                fill=True,  # можно изменить дизайн
                fill_color='#96C2DB',  # можно изменить дизайн
                fill_opacity=0.5,  # можно изменить дизайн
                tooltip=f"Адрес: {row['addr:street']} {row['addr:housenumber']}"
            )
        )

# добавляем группы на карту
m.add_child(parking_group)
m.add_child(charging_group)
m.add_child(buildings_group)

# добавляем контрольный слой для переключения между слоями
folium.LayerControl().add_to(m)

# отображаем карту
m


**Сохранение карты**

In [437]:
m.save('buildings_osm_result.html')

# Сервисы по площади




**Загрузка данных**

Выберите **один** из способов загрузки данных:
1. Данные из OpenStreetMap (OSM) - **вставьте название** своей территории в переменную "area_2";
2. Собственный файл - подгрузите Ваш файл в Jupyter Notebook, проверьте наличие **всех необходимых столбцов** в Вашем файле и обратите внимание на **название файла**, указанного в коде (либо назавите свой файл идентично, либо поменяйте название внутри кода)

При **повторном использовании** на другом сервисе:
Данные раздел подходит для расчёта таких сервисов как бизнес-центры и торговые центры.

При последовательном использовании данных сервисов рекомендуется использовать различные **наименования для переменных разных сервисов** (например, service_1 заменить на service_2)

Переменная **tags для бизнес-центров** (в случае использования данных из OSM) = {'building': 'office'} - стоит по умолчанию, также укажите свою территорию;

Переменная **tags для торговых центров** (в случае использования данных из OSM) = {'shop': 'mall'}, также укажите совю территорию

In [ ]:
 # укажите свою территорию в переменной area__2
area_2 = "Московский район, Санкт-Петербург, Россия"

# загружаем выбранный сервис (см. текст выше)
# для торгового центра поставьте наименование своей переменной
tags = {'building': 'office'}
service_1 = ox.geometries_from_place(area_2, tags)

In [401]:
# подгрузить файл с подготовленный сервисами с полями "парковочные_места_СИМ", "зарядные_места_СИМ", "Площадь", "addr:housenumber", "addr:street"
# название Вашего файла и название файла в коде должны совпадать
# для торгового центра поставьте наименование своей переменной
service_1 = gpd.read_file('/content/service_1_over.geojson')


In [ ]:
service_1 # проверка данных

In [ ]:
# картографическое представление данных
# для торгового центра поставьте наименование своей переменной
service_1.explore()

**Метод**

Запустите последовательно все части кода внутри подраздела "Метод". Убедитесь, что таблица атрибутов измениилась (или появилась информация о достаточности инфраструктуры для СИМ).

Текущая функция в коде подходит для бизнес-центров.

Обратите внимание на **изменения в коде** (в доп. подписях внутри функции ниже) коэффициента расчёта парковочных мест для СИМ для торговых центров:
- коэффициент для бизнес-центров: 0.004;
- коэффициент для торговых центров: 0.06.


In [439]:
def calculate_lots_service_1(geos_layer, result, count):
    # проверяем наличие необходимых атрибутов
    if all(attr in geos_layer.columns for attr in ['парковочные_места_СИМ', 'зарядные_места_СИМ', 'Площадь']):

        # проверяем минимальное необходимое количество мест
        required_parking = round(geos_layer['Площадь'] * 0.004) # для торговых центров: 0.06
        required_charging = round(required_parking * 0.1)
        # проверяем, более 50% объектов удовлетворяют условиям оценки
        satisfied_count_parking = (geos_layer['парковочные_места_СИМ'] >= required_parking).sum()
        satisfied_count_charging = (geos_layer['зарядные_места_СИМ'] >= required_charging).sum()
        total_count = len(geos_layer)

        parking_ratio = satisfied_count_parking / total_count
        charging_ratio = satisfied_count_charging / total_count

        if parking_ratio >= 0.5 and charging_ratio >= 0.5:
           result += 1
           count += 1
        elif parking_ratio <= 0.5 and charging_ratio >= 0.5:
            result += 0.5
            count += 1
        elif parking_ratio >= 0.5 and charging_ratio <= 0.5:
            result += 0.5
            count += 1
        else:
            count += 1

        print(f"Результат сервиса: {result}")
        print(f"Количество пройденных сервисов на территории: {count}")

        # обновляем парковочные места только если текущее значение меньше требуемого
        if (geos_layer['парковочные_места_СИМ'] < required_parking).any():
            geos_layer.loc[geos_layer['парковочные_места_СИМ'] < required_parking, 'парковочные_места_СИМ'] = required_parking
        else:
            print("Достаточное количество парковочных мест")

        # добавляем новое условие для проверки избыточности парковочных мест
        if ((geos_layer['парковочные_места_СИМ']) > required_parking + 20).any():
            print("Предупреждение: возможно избыточное количество парковочных мест")

        # обновляем зарядные места только если текущее значение меньше требуемого
        if (geos_layer['зарядные_места_СИМ'] < required_charging).any():
            geos_layer.loc[geos_layer['зарядные_места_СИМ'] < required_charging, 'зарядные_места_СИМ'] = required_charging
        else:
            print("Достаточное количество зарядных мест")

        # добавляем новое условие для проверки избыточности зарядных мест
        if ((geos_layer['зарядные_места_СИМ']) > required_parking + 15).any():
            print("Предупреждение: возможно избыточное количество зарядных мест")

    else:
        # обработка геослоя
        geos_layer = geos_layer[geos_layer.geometry.type.isin(['Polygon', 'MultiPolygon'])]
        geos_layer = geos_layer.reset_index(drop=True)
        original_crs = geos_layer.crs
        local_crs = geos_layer.estimate_utm_crs()  # определяем локальную систему координат
        geos_layer = geos_layer.to_crs(local_crs)  # переводим здания в локальную систему координат

        # преобразуем столбец 'building:levels' в числовой тип, ошибки будут заменены на NaN
        geos_layer['building:levels'] = pd.to_numeric(geos_layer['building:levels'], errors='coerce')
        # заполним пропущенные значения (NaN) нулями
        geos_layer = geos_layer.fillna(0)

        # добавляем или преобразуем необходимые атрибуты
        # 1. количество этажей (number_of_floors)
        geos_layer['number_of_floors'] = geos_layer.apply(
             lambda x: x['building:levels'] if x['building:levels'] > 1 else 1,
             axis=1
        )

        # 2. площадь застройки (footprint_area) - как площадь геометрии (основание здания)
        geos_layer['footprint_area'] = geos_layer.geometry.area

        # 3. общая площадь всех этажей (build_floor_area) - footprint_area * number_of_floors
        geos_layer['Площадь_2'] = geos_layer['footprint_area'] * geos_layer['number_of_floors']
        geos_layer['Площадь'] = geos_layer['Площадь_2'] * 0.4


        # теперь удалим все остальные столбцы, кроме 'geometry', 'build_floor_area', 'living_area',
        # 'footprint_area', 'number_of_floors', 'population', 'addr:housenumber', 'addr:street'
        geos_layer = geos_layer[['geometry', 'Площадь', 'footprint_area', 'number_of_floors', 'addr:housenumber', 'addr:street']]

        # расчёт парковочных и зарядных мест
        geos_layer['парковочные_места_СИМ'] = round(geos_layer['Площадь'] * 0.004) # для торговых центров: 0.06
        geos_layer['зарядные_места_СИМ'] = round(geos_layer['парковочные_места_СИМ'] * 0.1)

        # возвращаем к оригинальной системе координат
        geos_layer = geos_layer.to_crs(original_crs)

    return geos_layer, result, count

In [ ]:
 # для торгового центра поставьте наименование своей переменной
 # для торгового центра название функции: calculate_lots_service_2
service_1, result_service, count_service = calculate_lots_service_1(service_1, result_service, count_service)
service_1

**Сохренение файла**

In [ ]:
# для торгового центра поставьте наименование своей переменной
service_1.to_file('service_1_osm_result.geojson')

**Визуализация**

Запустите последовательно код в подразделе "Визуализация".

Дизайн карты (подложка, размер круговых маркеров, цвета объектов и т.д.) можно изменить по желанию.

In [ ]:
# для торгового центра поставьте наименование своей переменной (замените service_1 во всех случаях)
service_1['centroid'] = service_1.geometry.centroid

# добавление столбцов для широты и долготы центроидов
service_1['latitude'] = service_1['centroid'].y
service_1['longitude'] = service_1['centroid'].x

# удаление временного столбца 'centroid', если он не нужен
service_1 = service_1.drop(columns='centroid')


In [ ]:
# для торгового центра поставьте наименование своей переменной
# Создание DataFrame
df = pd.DataFrame(service_1)

# сипользуем средние значения для центра карты
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=12)

# создаем группы для парковочных и зарядных мест
parking_group = folium.FeatureGroup(name='Парковочные места')
charging_group = folium.FeatureGroup(name='Зарядные места')
buildings_group = folium.FeatureGroup(name='Сервис_1')

folium.TileLayer('CartoDB positron').add_to(m)

# итерация по строкам DataFrame
for idx, row in df.iterrows():
    # проверяем наличие необходимых колонок
    if 'парковочные_места_СИМ' in row and 'зарядные_места_СИМ' in row:
        # размер точки на основе количества парковочных мест
        parking_size = row['парковочные_места_СИМ'] / 10  # измените делитель по необходимости
        charging_places = row['зарядные_места_СИМ'] if row['зарядные_места_СИМ'] > 0 else 1

        # проверяем количество зарядных мест
        if charging_places >= 1:  # условие для зарядных мест
            charging_size = charging_places / 10  # измените делитель по необходимости

        # добавляем круговые маркеры для парковочных мест
        parking_group.add_child(
            folium.CircleMarker(
                location=(row['latitude'], row['longitude']),  # используем координаты текущей строки
                radius=parking_size,
                color=None,
                fill=True, # можно изменить дизайн
                fill_color='#BFA181', # можно изменить дизайн
                fill_opacity=0.6, # можно изменить дизайн
                tooltip=f"Парковочные места: {row['парковочные_места_СИМ']}"
            )
        )

        # добавляем круговые маркеры для зарядных мест
        charging_group.add_child(
            folium.CircleMarker(
                location=(row['latitude'], row['longitude']),  # Используем координаты текущей строки
                radius=charging_size,
                color=None,
                fill=True,  # можно изменить дизайн
                fill_color='#0A1828', # можно изменить дизайн
                fill_opacity=0.9, # можно изменить дизайн
                tooltip=f"Зарядные места: {row['зарядные_места_СИМ']}"  # Исправлено на нужное поле
            )
        )

# итерация для добавления зданий на карту
for index, row in df.iterrows():
    # проверяем наличие геометрии
    if 'geometry' in row and row['geometry'] is not None:
        geometry = row['geometry']

        # добавляем полигональный слой зданий на карту
        buildings_group.add_child(
            folium.GeoJson(
                geometry,  # используем всю геометрию
                color=None,
                fill=True, # можно изменить дизайн
                fill_color='#96C2DB', # можно изменить дизайн
                fill_opacity=0.5, # можно изменить дизайн
                tooltip=f"Адрес: {row['addr:street']} {row['addr:housenumber']}"
            )
        )

# добавляем группы на карту
m.add_child(parking_group)
m.add_child(charging_group)
m.add_child(buildings_group)

# добавляем контрольный слой для переключения между слоями
folium.LayerControl().add_to(m)

# отображаем карту
m

**Сохранение карты**

In [444]:
# название файла с картой можно заменить на свое. Особенно рекомендуется при повторном использовании кода на другом сервисе
m.save('service_1_osm_result.html')

# Сервисы по вместимости

**Загрузка данных**

Выберите **один** из способов загрузки данных:
1. Данные из OpenStreetMap (OSM) - **вставьте название** своей территории в переменную "area_3";
2. Собственный файл - подгрузите Ваш файл в Jupyter Notebook, проверьте наличие **всех необходимых столбцов** в Вашем файле и обратите внимание на **название файла**, указанного в коде (либо назавите свой файл идентично, либо поменяйте название внутри кода)

При **повторном использовании** на другом сервисе:
Данные раздел подходит для расчёта таких сервисов как бизнес-центры и торговые центры.

При последовательном использовании данных сервисов рекомендуется использовать различные **наименования для переменных разных сервисов** (например, service1 заменить на service2)

Переменная **tags для школ** (в случае использования данных из OSM) = {'building': 'school'} - стоит по умолчанию, также укажите свою территорию;

Переменная **tags для медицинских учреждений (поликлиник)** (в случае использования данных из OSM) = {'amenity': 'clinic'}, также укажите совю территорию

Переменная **tags для культурно-досуговых центров** (в случае использования данных из OSM) = {'amenity': 'community_centre'}, также укажите совю территорию

In [ ]:
  # укажите свою территорию в переменной area_3
area_3 = "Московский район, Санкт-Петербург, Россия"

# загружаем выбранный сервис (см. текст выше)
# для разных сервисов поставьте наименование своей переменной
tags = {'building': 'school'}
service1 = ox.geometries_from_place(area_3, tags)

In [408]:
  # подгрузить файл с подготовленный сервисами с полями "парковочные_места_СИМ", "зарядные_места_СИМ", "Вместимость", "addr:housenumber", "addr:street", "name"
  # название Вашего файла и название файла в коде должны совпадать
  # для разных сервисов поставьте наименование своей переменной
service1 = gpd.read_file('/content/service1_over.geojson')

In [ ]:
service1 # проверка данных

In [ ]:
 # картографическое представление данных
 # для разных сервисов поставьте наименование своей переменной
service1.explore()

**Метод**

Запустите последовательно все части кода внутри подраздела "Метод". Убедитесь, что таблица атрибутов измениилась (или появилась информация о достаточности инфраструктуры для СИМ).

Текущая функция в коде подходит для школ.

Обратите внимание на **изменения в коде** (в доп. подписях внутри функции ниже) коэффициентов расчёта парковочных мест для СИМ для поликлиник и культурно-досуговых центров:
- коэффициент для школ: 0.02;
- коэффициент для поликлиник: 0.01;
- коэффициент для культурно-досуговых центров: 0.1.

In [454]:
def calculate_lots_service1(geos_layer, result, count):
    # проверяем наличие необходимых атрибутов
    if all(attr in geos_layer.columns for attr in ['парковочные_места_СИМ', 'зарядные_места_СИМ', 'Вместимость']):

        # проверяем минимальное необходимое количество мест
        required_parking = round(geos_layer['Вместимость'] * 0.02)  # для поликлиник: 0.01; для культ-досуг центров: 0.1
        required_charging = round(required_parking * 0.1)
        # проверяем, более 50% объектов удовлетворяют условиям оценки
        satisfied_count_parking = (geos_layer['парковочные_места_СИМ'] >= required_parking).sum()
        satisfied_count_charging = (geos_layer['зарядные_места_СИМ'] >= required_charging).sum()
        total_count = len(geos_layer)

        parking_ratio = satisfied_count_parking / total_count
        charging_ratio = satisfied_count_charging / total_count

        if parking_ratio >= 0.5 and charging_ratio >= 0.5:
          result += 1
          count += 1
        elif parking_ratio <= 0.5 and charging_ratio >= 0.5:
            result += 0.5
            count += 1
        elif parking_ratio >= 0.5 and charging_ratio <= 0.5:
            result += 0.5
            count += 1
        else:
           count += 1

        print(f"Результат сервиса: {result}")
        print(f"Количество пройденных сервисов на территории: {count}")

        # боновляем парковочные места только если текущее значение меньше требуемого
        if (geos_layer['парковочные_места_СИМ'] < required_parking).any():
            geos_layer.loc[geos_layer['парковочные_места_СИМ'] < required_parking, 'парковочные_места_СИМ'] = required_parking
        else:
            print("Достаточное количество парковочных мест")

        # добавляем новое условие для проверки избыточности парковочных мест
        if ((geos_layer['парковочные_места_СИМ']) > required_parking + 20).any():
            print("Предупреждение: возможно избыточное количество парковочных мест")

        # обновляем зарядные места только если текущее значение меньше требуемого
        if (geos_layer['зарядные_места_СИМ'] < required_charging).any():
            geos_layer.loc[geos_layer['зарядные_места_СИМ'] < required_charging, 'зарядные_места_СИМ'] = required_charging
        else:
            print("Достаточное количество зарядных мест")

        # добавляем новое условие для проверки избыточности зарядных мест
        if ((geos_layer['зарядные_места_СИМ']) > required_parking + 15).any():
            print("Предупреждение: возможно избыточное количество зарядных мест")

    else:
    # обработка геослоя
       geos_layer = geos_layer[geos_layer.geometry.type.isin(['Polygon'])]
       geos_layer = geos_layer.reset_index(drop=True)
       original_crs = geos_layer.crs
       local_crs = geos_layer.estimate_utm_crs()
       geos_layer = geos_layer.to_crs(local_crs) # переводим школы в локальную систему координат
       geos_layer.crs  # переводим здания в локальную систему координат

       # преобразуем столбец 'building:levels' в числовой тип, ошибки будут заменены на NaN
       geos_layer['building:levels'] = pd.to_numeric(geos_layer['building:levels'], errors='coerce')

       # заполним пропущенные значения (NaN) нулями
       geos_layer = geos_layer.fillna(0)

       # добавляем или преобразуем необходимые атрибуты
       # 1. количество этажей (number_of_floors)
       geos_layer['number_of_floors'] = geos_layer.apply(
             lambda x: x['building:levels'] if x['building:levels'] > 1 else 1,
             axis=1
       )

       # 2. площадь застройки (footprint_area) - как площадь геометрии (основание здания)
       geos_layer['footprint_area'] = geos_layer.geometry.area

       # 3. общая площадь всех этажей (build_floor_area) - footprint_area * number_of_floors
       geos_layer['Площадь'] = geos_layer['footprint_area'] * geos_layer['number_of_floors']
       geos_layer['Расчетная_площадь'] = geos_layer['Площадь'] - (geos_layer['Площадь'] * 0.6) # для поликлинник: 0.5, для культ-досуг центров: 0.5

       geos_layer['Вместимость'] = geos_layer['Расчетная_площадь'] / 2.5   # для поликлинник: 4, для культ-досуг центров: 5

        # теперь удалим все остальные столбцы, кроме 'geometry', 'build_floor_area', 'living_area', 'footprint_area', 'number_of_floors', 'population'
       geos_layer = geos_layer[['geometry', 'name', 'Площадь', 'footprint_area', 'number_of_floors', 'addr:housenumber', 'addr:street', 'Расчетная_площадь', 'Вместимость']]

       # расчёт парковочных и зарядных мест
       geos_layer['парковочные_места_СИМ'] = round(geos_layer['Вместимость'] * 0.02) # для поликлиник: 0.01; для культ-досуг центров: 0.1
       geos_layer['зарядные_места_СИМ'] = round(geos_layer['парковочные_места_СИМ'] * 0.1)
       geos_layer = geos_layer.to_crs(original_crs)

    return geos_layer, result, count

In [ ]:
 # для разных сервисов поставьте наименование своей переменной
 # для поликлиник название функции: calculate_lots_service2
 # для культурно-досуговых центров название функции: calculate_lots_service3
service1, result_service, count_service = calculate_lots_service1(service1, result_service, count_service)
service1

**Сохранение файла**

In [ ]:
# для разных сервисов поставьте наименование своей переменной
service1.to_file('service1_osm_result.geojson')

**Визуализация**

Запустите последовательно код в подразделе "Визуализация".

Дизайн карты (подложка, размер круговых маркеров, цвета объектов и т.д.) можно изменить по желанию.

In [ ]:
# для разных сервисов поставьте наименование своей переменной (замените все service1)
service1['centroid'] = service1.geometry.centroid

# добавление столбцов для широты и долготы центроидов
service1['latitude'] = service1['centroid'].y
service1['longitude'] = service1['centroid'].x

# удаление временного столбца 'centroid', если он не нужен
service1 = service1.drop(columns='centroid')

In [ ]:
# создание DataFrame
df = pd.DataFrame(service1)

# используем средние значения для центра карты
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=12)

# создаем группы для парковочных и зарядных мест
parking_group = folium.FeatureGroup(name='Парковочные места')
charging_group = folium.FeatureGroup(name='Зарядные места')
buildings_group = folium.FeatureGroup(name='Сервис1')

folium.TileLayer('CartoDB positron').add_to(m)

# итерация по строкам DataFrame
for idx, row in df.iterrows():
    # проверяем наличие необходимых колонок
    if 'парковочные_места_СИМ' in row and 'зарядные_места_СИМ' in row:
        # размер точки на основе количества парковочных мест
        parking_size = row['парковочные_места_СИМ'] / 5  # измените делитель по необходимости
        charging_places = row['зарядные_места_СИМ'] if row['зарядные_места_СИМ'] > 0 else 1

        # проверяем количество зарядных мест
        if charging_places >= 1:  # условие для зарядных мест
            charging_size = charging_places / 5  # измените делитель по необходимости

        # добавляем круговые маркеры для парковочных мест
        parking_group.add_child(
            folium.CircleMarker(
                location=(row['latitude'], row['longitude']),  # используем координаты текущей строки
                radius=parking_size,
                color=None,
                fill=True,  # можно изменить дизайн
                fill_color='#BFA181',  # можно изменить дизайн
                fill_opacity=0.6,  # можно изменить дизайн
                tooltip=f"Парковочные места: {row['парковочные_места_СИМ']}"
            )
        )

        # добавляем круговые маркеры для зарядных мест
        charging_group.add_child(
            folium.CircleMarker(
                location=(row['latitude'], row['longitude']),  # используем координаты текущей строки
                radius=charging_size,
                color=None,
                fill=True,  # можно изменить дизайн
                fill_color='#0A1828',  # можно изменить дизайн
                fill_opacity=0.9,  # можно изменить дизайн
                tooltip=f"Зарядные места: {row['зарядные_места_СИМ']}"  # исправлено на нужное поле
            )
        )

# итерация для добавления зданий на карту
for index, row in df.iterrows():
    # проверяем наличие геометрии
    if 'geometry' in row and row['geometry'] is not None:
        geometry = row['geometry']

        # добавляем полигональный слой зданий на карту
        buildings_group.add_child(
            folium.GeoJson(
                geometry,  # используем всю геометрию
                color=None,
                fill=True,  # можно изменить дизайн
                fill_color='#96C2DB',  # можно изменить дизайн
                fill_opacity=0.5,  # можно изменить дизайн
                tooltip=f"Сервис1:{row['name']}"
            )
        )

# добавляем группы на карту
m.add_child(parking_group)
m.add_child(charging_group)
m.add_child(buildings_group)

# добавляем контрольный слой для переключения между слоями
folium.LayerControl().add_to(m)

# отображаем карту
m

**Сохранение карты**

In [460]:
 # название файла с картой можно заменить на свое. Особенно рекомендуется при повторном использовании кода на другом сервисе
m.save('service1_osm_result.html')

# Остановки наземного танспорта

**Загрузка данных**

Выберите **один** из способов загрузки данных:
1. Данные из OpenStreetMap (OSM) - **вставьте название** своей территории в переменную "area_4";
2. Собственный файл - подгрузите Ваш файл в Jupyter Notebook, проверьте наличие **всех необходимых столбцов** в Вашем файле и обратите внимание на **название файла**, указанного в коде (либо назавите свой файл идентично, либо поменяйте название внутри кода)

In [ ]:
 # укажите свою территорию в переменной area_4
area_4 = "Московский район, Санкт-Петербург, Россия"

# загружаем остановки
tags = {'highway': 'bus_stop'}
stops = ox.geometries_from_place(area_4, tags)


In [415]:
   # подгрузить файл с подготовленный точечными объектами с полями "парковочные_места_СИМ", "зарядные_места_СИМ"
   # название Вашего файла и название файла в коде должны совпадать
stops = gpd.read_file('/content/stops_over.geojson')

In [ ]:
stops # проверяем данные

In [ ]:
 # картографическое представление данных
stops.explore()

**Метод**

Запустите последовательно все части кода внутри подраздела "Метод". Убедитесь, что таблица атрибутов измениилась (или появилась информация о достаточности инфраструктуры для СИМ)

In [462]:
def calculate_stops(geos_layer, result, count):
    # проверяем наличие необходимых атрибутов
    if all(attr in geos_layer.columns for attr in ['парковочные_места_СИМ', 'зарядные_места_СИМ']):

        # заполняем NaN значения нулями, чтобы избежать ошибок
        geos_layer['парковочные_места_СИМ'] = geos_layer['парковочные_места_СИМ'].fillna(0)
        geos_layer['зарядные_места_СИМ'] = geos_layer['зарядные_места_СИМ'].fillna(0)
        # проверяем, более 50% объектов удовлетворяют условиям оценки
        satisfied_count_parking = (geos_layer['парковочные_места_СИМ'] >= 5).sum()
        satisfied_count_charging = (geos_layer['зарядные_места_СИМ'] >= 1).sum()
        total_count = len(geos_layer)

        parking_ratio = satisfied_count_parking / total_count
        charging_ratio = satisfied_count_charging / total_count

        if parking_ratio >= 0.5 and charging_ratio >= 0.5:
          result += 1
          count += 1
        elif parking_ratio <= 0.5 and charging_ratio >= 0.5:
            result += 0.5
            count += 1
        elif parking_ratio >= 0.5 and charging_ratio <= 0.5:
            result += 0.5
            count += 1
        else:
           count += 1

        print(f"Результат сервиса: {result}")
        print(f"Количество пройденных сервисов на территории: {count}")

        # проверяем минимальное необходимое количество парковочных мест
        if (geos_layer['парковочные_места_СИМ'] >= 5).all():
            # если количество парковочных мест достаточно, ничего не делаем
            print("Достаточное количество парковочных мест для СИМ")
        # добавляем новое условие для проверки избыточности парковочных мест
        elif (geos_layer['парковочные_места_СИМ'] > 20).any():
            print("Предупреждение: возможно избыточное количество парковочных мест")
        else:
            geos_layer['парковочные_места_СИМ'] = 5

        if (geos_layer['зарядные_места_СИМ'] >= 1).all():
            # если количество зарядных мест достаточно, ничего не делаем
            print("Достаточное количество зарядных мест для СИМ")
         # добавляем новое условие для проверки избыточности зарядных мест
        elif (geos_layer['зарядные_места_СИМ'] > 7).any():
            print("Предупреждение: возможно избыточное количество зарядных мест")
        else:
            geos_layer['зарядные_места_СИМ'] = 1

    else:
        # обработка геослоя
        geos_layer = geos_layer.reset_index(drop=True)
        original_crs = geos_layer.crs
        local_crs = geos_layer.estimate_utm_crs()
        geos_layer = geos_layer.to_crs(local_crs)  # переводим в локальную систему координат

        # заполняем NaN значения нулями
        geos_layer = geos_layer.fillna(0)

        # расчёт парковочных и зарядных мест
        geos_layer['парковочные_места_СИМ'] = 5
        geos_layer['зарядные_места_СИМ'] = 1
        # теперь удалим все остальные столбцы, кроме 'geometry', 'name', 'парковочные_места_СИМ', 'зарядные_места_СИМ'
        geos_layer = geos_layer[['geometry', 'name', 'парковочные_места_СИМ', 'зарядные_места_СИМ']]
        geos_layer = geos_layer.to_crs(original_crs)

    return geos_layer, result, count

In [ ]:
stops, result_service, count_service = calculate_stops(stops, result_service, count_service)
stops

**Сохранение файла**



In [464]:
stops.to_file('stops_osm_result.geojson')

**Визуализация**

Запустите последовательно код в подразделе "Визуализация".

Дизайн карты (подложка, размер круговых маркеров, цвета объектов и т.д.) можно изменить по желанию.

In [ ]:
stops['centroid'] = stops.geometry.centroid

# добавление столбцов для широты и долготы центроидов
stops['latitude'] = stops['centroid'].y
stops['longitude'] = stops['centroid'].x

# удаление временного столбца 'centroid', если он не нужен
stops = stops.drop(columns='centroid')

In [ ]:
# создание DataFrame
df = pd.DataFrame(stops)

# используем средние значения для центра карты
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=12)

# создаем группы для парковочных и зарядных мест
parking_group = folium.FeatureGroup(name='Парковочные места')
charging_group = folium.FeatureGroup(name='Зарядные места')

folium.TileLayer('CartoDB positron').add_to(m)

# итерация по строкам DataFrame
for idx, row in df.iterrows():
    # проверяем наличие необходимых колонок и заполняем NaN значения нулями
    parking_spots = row.get('парковочные_места_СИМ', 0)  # если нет, то 0
    charging_spots = row.get('зарядные_места_СИМ', 0)  # если нет, то 0

    # размер точки на основе количества парковочных мест
    parking_size = max(parking_spots / 1, 1)  # минимальный размер точки 1
    charging_size = max(charging_spots / 1, 1) if charging_spots > 0 else 1  # минимальный размер точки 1

    # добавляем круговые маркеры для парковочных мест
    parking_group.add_child(
        folium.CircleMarker(
            location=(row['latitude'], row['longitude']),
            radius=parking_size,
            color=None,
            fill=True,  # можно изменить дизайн
            fill_color='#BFA181', # можно изменить дизайн
            fill_opacity=0.6,  # можно изменить дизайн
            tooltip=f"Парковочные места: {parking_spots}"
        )
    )

    # добавляем круговые маркеры для зарядных мест
    charging_group.add_child(
        folium.CircleMarker(
            location=(row['latitude'], row['longitude']),
            radius=charging_size,
            color=None,
            fill=True,  # можно изменить дизайн
            fill_color='#0A1828',  # можно изменить дизайн
            fill_opacity=0.9, # можно изменить дизайн
            tooltip=f"Зарядные места: {charging_spots}"
        )
    )

# добавляем группы на карту
m.add_child(parking_group)
m.add_child(charging_group)

# добавляем контрольный слой для переключения между слоями
folium.LayerControl().add_to(m)

# отображаем карту
m

**Сохранение карты**

In [467]:
m.save('stops_osm_result.html')

# Станции метро

**Загрузка данных**

Выберите **один** из способов загрузки данных:
1. Данные из OpenStreetMap (OSM) - **вставьте название** своей территории в переменную "area_5" - первый код;
2. Собственный файл - подгрузите Ваш файл в Jupyter Notebook, проверьте наличие **всех необходимых столбцов** в Вашем файле и обратите внимание на **название файла**, указанного в коде (либо назавите свой файл идентично, либо поменяйте название внутри кода) - второй код

In [ ]:
 # укажите свою территорию в переменной area_5
area_5 = "Московский район, Санкт-Петербург, Россия"

# загружаем станции метро
tags = {'railway': 'station'}
metro = ox.geometries_from_place(area_5, tags)
gdf = gpd.GeoDataFrame(metro)

# удаление объектов, не равных "ГУП «Петербургский метрополитен»" в колонке "operator"
gdf_filtered = gdf[gdf['operator'] == 'ГУП «Петербургский метрополитен»']

# вывод отфильтрованного GeoDataFrame
metro=gdf_filtered

In [422]:
 # подгрузить файл с подготовленный точечными объектами с полями "парковочные_места_СИМ", "зарядные_места_СИМ"
 # название Вашего файла и название файла в коде должны совпадать
metro = gpd.read_file('/content/metro_over.geojson')

In [ ]:
metro # проверка данных

In [ ]:
 # картографическое представление данных
metro.explore()

**Метод**

Запустите последовательно все части кода внутри подраздела "Метод". Убедитесь, что таблица атрибутов измениилась (или появилась информация о достаточности инфраструктуры для СИМ)

In [423]:
def calculate_metro(geos_layer, result, count):
    # проверяем наличие необходимых атрибутов
    if all(attr in geos_layer.columns for attr in ['парковочные_места_СИМ', 'зарядные_места_СИМ']):

        # заполняем NaN значения нулями, чтобы избежать ошибок
        geos_layer['парковочные_места_СИМ'] = geos_layer['парковочные_места_СИМ'].fillna(0)
        geos_layer['зарядные_места_СИМ'] = geos_layer['зарядные_места_СИМ'].fillna(0)
        # проверяем, более 50% объектов удовлетворяют условиям оценки
        satisfied_count_parking = (geos_layer['парковочные_места_СИМ'] >= 20).sum()
        satisfied_count_charging = (geos_layer['парковочные_места_СИМ'] * 0.1).sum()
        total_count = len(geos_layer)

        parking_ratio = satisfied_count_parking / total_count
        charging_ratio = satisfied_count_charging / total_count

        if parking_ratio >= 0.5 and charging_ratio >= 0.5:
          result += 1
          count += 1
        elif parking_ratio <= 0.5 and charging_ratio >= 0.5:
            result += 0.5
            count += 1
        elif parking_ratio >= 0.5 and charging_ratio <= 0.5:
            result += 0.5
            count += 1
        else:
           count += 1

        print(f"Результат сервиса: {result}")
        print(f"Количество пройденных сервисов на территории: {count}")

        # проверяем минимальное необходимое количество зарядных мест
        if (geos_layer['парковочные_места_СИМ'] >= 20).all():
            # если количество парковочных мест достаточно, ничего не делаем
            print("Достаточное количество парковочных мест для СИМ")
        elif (geos_layer['парковочные_места_СИМ'] > 70).any():
            print("Предупреждение: возможно избыточное количество парковочных мест")
        else:
            geos_layer['парковочные_места_СИМ'] = 20

        if (geos_layer['зарядные_места_СИМ'] >= (geos_layer['парковочные_места_СИМ'] * 0.1)).all():
            # если количество зарядных мест достаточно, ничего не делаем
            print("Достаточное количество зарядных мест для СИМ")
        elif (geos_layer['зарядные_места_СИМ'] >= ((geos_layer['парковочные_места_СИМ'] * 0.1)+20)).any():
            print("Предупреждение: возможно избыточное количество зарядных мест")
        else:
            geos_layer['зарядные_места_СИМ'] = round(geos_layer['парковочные_места_СИМ'] * 0.1)

    else:
        # обработка геослоя
        geos_layer = geos_layer.reset_index(drop=True)
        original_crs = geos_layer.crs
        local_crs = geos_layer.estimate_utm_crs()
        geos_layer = geos_layer.to_crs(local_crs)  # переводим в локальную систему координат

        # заполняем NaN значения нулями
        geos_layer = geos_layer.fillna(0)

        # расчёт парковочных и зарядных мест
        geos_layer['парковочные_места_СИМ'] = 20
        geos_layer['зарядные_места_СИМ'] = round(geos_layer['парковочные_места_СИМ'] * 0.1)
        # теперь удалим все остальные столбцы, кроме 'geometry', 'name', 'парковочные_места_СИМ', 'зарядные_места_СИМ'
        geos_layer = geos_layer[['geometry', 'name', 'парковочные_места_СИМ', 'зарядные_места_СИМ']]
        geos_layer = geos_layer.to_crs(original_crs)

    return geos_layer, result, count

In [ ]:
metro, result_service, count_service = calculate_metro(metro, result_service, count_service)
metro

**Сохранение файла**

In [470]:
metro.to_file('metro_osm_result.geojson')

**Визуализация**

Запустите последовательно код в подразделе "Визуализация".

Дизайн карты (подложка, размер круговых маркеров, цвета объектов и т.д.) можно изменить по желанию.

In [ ]:
metro['centroid'] = metro.geometry.centroid

# добавление столбцов для широты и долготы центроидов
metro['latitude'] = metro['centroid'].y
metro['longitude'] = metro['centroid'].x

# удаление временного столбца 'centroid', если он не нужен
metro = metro.drop(columns='centroid')

In [ ]:
# создание DataFrame
df = pd.DataFrame(metro)

# используем средние значения для центра карты
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=12)

# создаем группы для парковочных и зарядных мест
parking_group = folium.FeatureGroup(name='Парковочные места')
charging_group = folium.FeatureGroup(name='Зарядные места')

folium.TileLayer('CartoDB positron').add_to(m)

# итерация по строкам DataFrame
for idx, row in df.iterrows():
    # проверяем наличие необходимых колонок и заполняем NaN значения нулями
    parking_spots = row.get('парковочные_места_СИМ', 0)  # если нет, то 0
    charging_spots = row.get('зарядные_места_СИМ', 0)  # если нет, то 0

    # размер точки на основе количества парковочных мест
    parking_size = max(parking_spots / 1, 1)  # минимальный размер точки 1
    charging_size = max(charging_spots / 1, 1) if charging_spots > 0 else 1  # минимальный размер точки 1

    # добавляем круговые маркеры для парковочных мест
    parking_group.add_child(
        folium.CircleMarker(
            location=(row['latitude'], row['longitude']),
            radius=parking_size,
            color=None,
            fill=True,  # можно изменить дизайн
            fill_color='#BFA181',  # можно изменить дизайн
            fill_opacity=0.6, # можно изменить дизайн
            tooltip=f"Парковочные места: {parking_spots}"
        )
    )

    # добавляем круговые маркеры для зарядных мест
    charging_group.add_child(
        folium.CircleMarker(
            location=(row['latitude'], row['longitude']),
            radius=charging_size,
            color=None,
            fill=True,  # можно изменить дизайн
            fill_color='#0A1828',  # можно изменить дизайн
            fill_opacity=0.9,  # можно изменить дизайн
            tooltip=f"Зарядные места: {charging_spots}"
        )
    )

# добавляем группы на карту
m.add_child(parking_group)
m.add_child(charging_group)

# добавляем контрольный слой для переключения между слоями
folium.LayerControl().add_to(m)

# отображаем карту
m

**Сохранение карты**

In [473]:
m.save('metro_osm_result.html')

# Оценка территории

Используйте этот раздел, если Вы работали с собственными файлами (не с данными из OSM)

In [ ]:
 # используйте после проверки всех сервисов на Вашей территории
terr_point = result_service/count_service
if terr_point < 0.4:
  print ("Оценка: низкая степень интеграции инфраструктуры для средств микромобильности на Вашей территории")
elif 0.4 <= terr_point < 0.7:
  print ("Оценка: умеренная степень интеграции инфраструктуры для средств микромобильности на Вашей территории")
elif terr_point >= 0.7:
  print ("Оценка: высокая степень интеграции инфраструктуры для средств микромобильности на Вашей территории")
terr_point